In [1]:
import matplotlib
import torchvision
import torch
import torch.nn as nn
import numpy as np 
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
from einops import rearrange, repeat
%matplotlib inline
from thop import profile

def show_macs_params(net, a):
    macs, params = profile(net, inputs=(a,), verbose=False)  # macs == FLOPS, GFLOPS == 1e12 * FLOPS
    print(f"{net._get_name()}\t\t{macs=}\t{params=}")
    from thop import clever_format
    macs, params = clever_format([macs, params], "%.3f")
    print(f"{macs=}\t{params=}\n")

In [ ]:

from models.hourglass_SA import HourglassNet_SA as Network
# from models.lite_hrnet import LiteHRNet as Network
device = torch.device('cpu')

state = torch.load('checkpoint/Center_SimDR/1HG-ME-att-c128-k2/73.82_AP_0epoch.pt', map_location=device)['state_dict']
total_params = sum([v.nelement() for v in state.values()])
print("Number of parameter: %.2fM" % (total_params / 1e6))

for k, v in state.items():
    print(f"{k}:\t {v.shape}")

a = torch.rand(1, 3, 256, 256)
net = Network().to(device)
# 通过 thop 计算
show_macs_params(net, a)

In [3]:
size = 64
class SimDR(nn.Module):

    def __init__(self) :
        super(SimDR, self).__init__()
        
        self.pred_x = nn.Linear(size ** 2 , int(256 * 2)) 
        self.pred_y = nn.Linear(size ** 2, int(256 * 2))

    def forward(self, x):
        # kpts = self.vector_feature(feature)
        # kpts = rearrange(kpts, 'b c h w -> b c (h w)')
        b,c, h, w = x.shape
        x = x.view(b, c, -1)
        pred_x = self.pred_x(x)  # (b, c, w * k)
        pred_y = self.pred_y(x)  # (b, c, h * k)
        return pred_x, pred_y  

from thop import profile
from models.layers import LiteHG, LiteResidual, Residual, LiteBRC, InvertedResidual, GhostConv, DWConv, SplitDWConv, channel_attention3x3, GhostConv, ReducedConv1x1, ms_att, fuse_block
from models.LiteHourglass import BRC, StemBlock, LiteHourglass
from models.attention import NAM_Channel_Att
from models.hourglass_SA import ME_att, HourglassNet_SA, my_pelee_stem

print('successfully import package')

a = torch.rand(1, 21, size, size)
show_macs_params(SimDR(), a)

a = torch.rand(1, 3, 256, 256)
show_macs_params(StemBlock(64, 32), a)


size = 32
c = 128
a = torch.rand(1, c, size, size)

# net = NAM_Channel_Att(c)
# net = channel_attention3x3(c)
# net = InvertedResidual(c, c, 2)
show_macs_params(DWConv(c, c), a)
# net = GhostConv(c, c)
show_macs_params(SplitDWConv(c, c), a)
show_macs_params(channel_attention3x3(c), a)

# show_macs_params(LiteHG(4, c), a)
# net = Bi_HG(4, c)
# net = BRC(c, c)
# net = GhostConv(c, c)
# net = ReducedConv1x1(c, c)

show_macs_params(ME_att(c, c), a)
# net = ms_att(c)


# net = MS_HG(c, c * 4, 4, 4)
# net = LiteResidual(c, c)
# show_macs_params(net, a)

# net = LiteBRC(c, c)

# net = ME_att(c, c)

show_macs_params(BRC(c, c), a)
show_macs_params(Residual(c, c), a)
# # y = net(a)

print(f"完整网络的参数：")
a = torch.rand(1, 3, 128, 128)
# net = my_pelee_stem(128)
# net = LiteHourglass(1, 128)
net = HourglassNet_SA(1)
show_macs_params(net, a)

successfully import package
SimDR		macs=88080384.0	params=4195328.0
macs='88.080M'	params='4.195M'

StemBlock		macs=96075776.0	params=16000.0
macs='96.076M'	params='16.000K'

DWConv		macs=18481152.0	params=18048.0
macs='18.481M'	params='18.048K'

SplitDWConv		macs=5246656.0	params=27904.0
macs='5.247M'	params='27.904K'

channel_attention3x3		macs=136320.0	params=18048.0
macs='136.320K'	params='18.048K'

ME_att		macs=54727808.0	params=71360.0
macs='54.728M'	params='71.360K'

BRC		macs=151257088.0	params=147712.0
macs='151.257M'	params='147.712K'

Residual		macs=55312384.0	params=54016.0
macs='55.312M'	params='54.016K'

完整网络的参数：
HourglassNet_SA		macs=301706880.0	params=5043736.0
macs='301.707M'	params='5.044M'



In [ ]:
class Stem(nn.Module):
    def __init__(self, inp_dim=128) :
        super().__init__() 
        self.pre = nn.Sequential(
            # Conv(3, 64, 7, 2, bn=True, relu=True),
            nn.Conv2d(3, 64, 7, 2, 3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            Residual(64, 128),
            nn.MaxPool2d(2, 2),
            Residual(128, 128),
            Residual(128, inp_dim)
        )
    def forward(self, x):
        return self.pre(x)

a = torch.rand(1, 3, 256, 256)
net = Stem()
show_macs_params(net, a)

In [21]:
class channel_attention3x3(nn.Module):
    def __init__(self, channel=128):
        super().__init__()
        self.att = nn.Sequential(
                    nn.AdaptiveAvgPool2d((1, 1)),
                    nn.BatchNorm2d(channel),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(channel, channel, 1, 1, 0, groups=channel),      
                    # nn.Dropout(p=0.3),
                    nn.Flatten(),
                    # nn.Linear(channel, channel),
                    nn.Sigmoid(),  
                    # nn.BatchNorm1d(channel),
                    # nn.LeakyReLU(inplace=True),  
                    )
        

    def forward(self, x):
        att = self.att(x)
        b, c, _, _ = x.shape
        x = x * att.view(b, c, 1, 1)
        return x

a = torch.rand(1, 128, 64, 64)
net = channel_attention3x3()
show_macs_params(net, a)

channel_attention3x3		macs=524928.0	params=512.0
macs='524.928K'	params='512.000B'



In [7]:
from torch.nn import functional as F
a = torch.rand(1, 2, 5, 5)
# a = F.upsample_nearest(a, scale_factor=2)
a= F.interpolate(a, scale_factor=2, mode='nearest')
a.shape

torch.Size([1, 2, 10, 10])

In [10]:
np.random.rand(1) > 0.5

array([ True])

In [26]:
a = np.random.rand(1) 
if a > 0.5:
    print(a, True)
else:
    print(a, False)

[0.29818155] False
